<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt)_MondayOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.
#### 엑셀로 저장할 때 파라미터를 제외한 문구들(맨 위에 화면구성, 맨 아래에 주석 등)을 지우고 작업하는게 편합니다

In [2]:
# 파일 업로드
# 예제파일 2개(이론가파람_SK하이닉스, 상하한가) 파일을 https://github.com/SophieShin/MachineLearning/tree/main 에서 다운 받을 수 있습니다.

from google.colab import files

uploaded = files.upload()

Saving 월요일옵션0728.csv to 월요일옵션0728.csv


#### 아래 uploaded['이론가파람_SK하이닉스.csv'] 부분에서 한글 부분을 자신이 저장한 파일 이름으로 변경합니다

In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['월요일옵션0728.csv']), encoding='utf-8', thousands = ',')
df

,종목명,Unnamed: 1,선물옵션\n구분,Unnamed: 3,최종\n거래일,Unnamed: 5,행사가격,Unnamed: 7,기초자산\n기준가격,Unnamed: 9,...,Unnamed: 15,배당 현재가치,Unnamed: 17,변동성,Unnamed: 19,Unnamed: 20,이론가격,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,C 2308W1 305.0,NaN,콜,NaN,20230807.0,NaN,305.0,NaN,344.03,NaN,...,NaN,0.0,NaN,0.121,NaN,NaN,39.28,NaN,NaN,NaN
1,C 2308W1 307.5,NaN,콜,NaN,20230807.0,NaN,307.5,NaN,344.03,NaN,...,NaN,0.0,NaN,0.121,NaN,NaN,36.78,NaN,NaN,NaN
2,C 2308W1 310.0,NaN,콜,NaN,20230807.0,NaN,310.0,NaN,344.03,NaN,...,NaN,0.0,NaN,0.121,NaN,NaN,34.28,NaN,NaN,NaN
3,C 2308W1 312.5,NaN,콜,NaN,20230807.0,NaN,312.5,NaN,344.03,NaN,...,NaN,0.0,NaN,0.121,NaN,NaN,31.79,NaN,NaN,NaN
4,C 2308W1 315.0,NaN,콜,NaN,20230807.0,NaN,315.0,NaN,344.03,NaN,...,NaN,0.0,NaN,0.121,NaN,NaN,29.29,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,P 2308W1 385.0,NaN,풋,NaN,20230807.0,NaN,385.0,NaN,344.03,NaN,...,NaN,0.0,NaN,0.141,NaN,NaN,40.65,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,"* : ETF선물의 경우, 분배금의 미래가치",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [5]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

<ipython-input-5-6bf907c65afb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['금리(%)']= df['금리(%)']/100


In [8]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션\n구분'] == '풋']

In [9]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션\n구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [10]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,C 2308W1 305.0,콜,20230807.0,305.0,344.03,8.0,0.0374,0.0,0.121,39.28
1,C 2308W1 307.5,콜,20230807.0,307.5,344.03,8.0,0.0374,0.0,0.121,36.78
2,C 2308W1 310.0,콜,20230807.0,310.0,344.03,8.0,0.0374,0.0,0.121,34.28
3,C 2308W1 312.5,콜,20230807.0,312.5,344.03,8.0,0.0374,0.0,0.121,31.79
4,C 2308W1 315.0,콜,20230807.0,315.0,344.03,8.0,0.0374,0.0,0.121,29.29
5,C 2308W1 317.5,콜,20230807.0,317.5,344.03,8.0,0.0374,0.0,0.121,26.79
6,C 2308W1 320.0,콜,20230807.0,320.0,344.03,8.0,0.0374,0.0,0.121,24.29
7,C 2308W1 322.5,콜,20230807.0,322.5,344.03,8.0,0.0374,0.0,0.121,21.79
8,C 2308W1 325.0,콜,20230807.0,325.0,344.03,8.0,0.0374,0.0,0.121,19.30
9,C 2308W1 327.5,콜,20230807.0,327.5,344.03,8.0,0.0374,0.0,0.121,16.80


In [11]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(float,df['기초자산\n기준가격'].tolist()))
K = list(map(float,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(float,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [12]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*1.2
lower_lmt =  df['기초자산\n기준가격']*0.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [13]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [14]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [15]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [16]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [17]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [18]:
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,C 2308W1 305.0,콜,20230807.0,305.0,344.03,8.0,0.0374,0.0,0.121,39.28,3.927991e+01,8.600614e-05,107.249914,0.0
1,C 2308W1 307.5,콜,20230807.0,307.5,344.03,8.0,0.0374,0.0,0.121,36.78,3.678196e+01,-1.962469e-03,104.751962,0.0
2,C 2308W1 310.0,콜,20230807.0,310.0,344.03,8.0,0.0374,0.0,0.121,34.28,3.428401e+01,-4.010945e-03,102.254011,0.0
3,C 2308W1 312.5,콜,20230807.0,312.5,344.03,8.0,0.0374,0.0,0.121,31.79,3.178606e+01,3.940576e-03,99.756059,0.0
4,C 2308W1 315.0,콜,20230807.0,315.0,344.03,8.0,0.0374,0.0,0.121,29.29,2.928811e+01,1.891995e-03,97.258108,0.0
5,C 2308W1 317.5,콜,20230807.0,317.5,344.03,8.0,0.0374,0.0,0.121,26.79,2.679016e+01,-1.577510e-04,94.760156,0.0
6,C 2308W1 320.0,콜,20230807.0,320.0,344.03,8.0,0.0374,0.0,0.121,24.29,2.429222e+01,-2.222344e-03,92.262205,0.0
7,C 2308W1 322.5,콜,20230807.0,322.5,344.03,8.0,0.0374,0.0,0.121,21.79,2.179438e+01,-4.376394e-03,89.764253,0.0
8,C 2308W1 325.0,콜,20230807.0,325.0,344.03,8.0,0.0374,0.0,0.121,19.30,1.929717e+01,2.831205e-03,87.266302,0.0
9,C 2308W1 327.5,콜,20230807.0,327.5,344.03,8.0,0.0374,0.0,0.121,16.80,1.680234e+01,-2.338045e-03,84.768350,0.0


In [19]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [20]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
33,P 2308W1 305.0,풋,20230807.0,305.0,344.03,8.0,0.0374,0.0,0.141,0.00
34,P 2308W1 307.5,풋,20230807.0,307.5,344.03,8.0,0.0374,0.0,0.141,0.00
35,P 2308W1 310.0,풋,20230807.0,310.0,344.03,8.0,0.0374,0.0,0.141,0.00
36,P 2308W1 312.5,풋,20230807.0,312.5,344.03,8.0,0.0374,0.0,0.141,0.00
37,P 2308W1 315.0,풋,20230807.0,315.0,344.03,8.0,0.0374,0.0,0.141,0.00
38,P 2308W1 317.5,풋,20230807.0,317.5,344.03,8.0,0.0374,0.0,0.141,0.00
39,P 2308W1 320.0,풋,20230807.0,320.0,344.03,8.0,0.0374,0.0,0.141,0.00
40,P 2308W1 322.5,풋,20230807.0,322.5,344.03,8.0,0.0374,0.0,0.141,0.00
41,P 2308W1 325.0,풋,20230807.0,325.0,344.03,8.0,0.0374,0.0,0.141,0.00
42,P 2308W1 327.5,풋,20230807.0,327.5,344.03,8.0,0.0374,0.0,0.141,0.02


In [21]:
#인덱스 초기화
df2.reset_index(inplace = True)

In [22]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(float,df2['기초자산\n기준가격'].tolist()))
K = list(map(float,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(float,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [23]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*0.8
lower_lmt =  df['기초자산\n기준가격']*1.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [24]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [25]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [26]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [27]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [28]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [29]:
df2

,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,33,P 2308W1 305.0,풋,20230807.0,305.0,344.03,8.0,0.0374,0.0,0.141,0.00,2.514772e-10,-2.514772e-10,29.774963,0.0
1,34,P 2308W1 307.5,풋,20230807.0,307.5,344.03,8.0,0.0374,0.0,0.141,0.00,4.696678e-09,-4.696678e-09,32.261217,0.0
2,35,P 2308W1 310.0,풋,20230807.0,310.0,344.03,8.0,0.0374,0.0,0.141,0.00,6.230179e-08,-6.230179e-08,34.753053,0.0
3,36,P 2308W1 312.5,풋,20230807.0,312.5,344.03,8.0,0.0374,0.0,0.141,0.00,9.514823e-07,-9.514823e-07,37.247239,0.0
4,37,P 2308W1 315.0,풋,20230807.0,315.0,344.03,8.0,0.0374,0.0,0.141,0.00,7.412911e-06,-7.412911e-06,39.743446,0.0
5,38,P 2308W1 317.5,풋,20230807.0,317.5,344.03,8.0,0.0374,0.0,0.141,0.00,4.697370e-05,-4.697370e-05,42.240612,0.0
6,39,P 2308W1 320.0,풋,20230807.0,320.0,344.03,8.0,0.0374,0.0,0.141,0.00,3.063217e-04,-3.063217e-04,44.738107,0.0
7,40,P 2308W1 322.5,풋,20230807.0,322.5,344.03,8.0,0.0374,0.0,0.141,0.00,1.374073e-03,-1.374073e-03,47.235882,0.0
8,41,P 2308W1 325.0,풋,20230807.0,325.0,344.03,8.0,0.0374,0.0,0.141,0.00,4.997572e-03,-4.997572e-03,49.733758,0.0
9,42,P 2308W1 327.5,풋,20230807.0,327.5,344.03,8.0,0.0374,0.0,0.141,0.02,1.822482e-02,1.775182e-03,52.231670,0.0


In [30]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


In [ ]:
df2.to_excel('미니옵션풋_검증용.xlsx')